# config

In [ ]:
from getpass import getpass
import requests, json

token = getpass("Enter GitHub token (hidden): ")

r = requests.get("https://api.github.com/user", headers={"Authorization": f"token {token}"})
print(r.status_code)
print(r.json().get("login"))

Enter GitHub token (hidden): ··········
200
Vu-Quoc-Tuan


In [ ]:
username = "Vu-Quoc-Tuan"
repo_name = "BTL-NLP-2526I_INT3406_3"

# Sử dụng biến token để xác thực trực tiếp trong URL
!git clone https://{token}@github.com/{username}/{repo_name}

Cloning into 'BTL-NLP-2526I_INT3406_3'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 67 (delta 23), reused 33 (delta 12), pack-reused 23 (from 2)
Receiving objects: 100% (67/67), 47.83 MiB | 12.73 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [ ]:
# 1) vào thư mục repo (thay path nếu khác)
%cd /content/BTL-NLP-2526I_INT3406_3

# 2) kiểm tra hiện tại đang ở đâu và liệt kê file
!pwd
!ls -la
!ls -la vlsp-mt
!test -f vlsp-mt/requirements.txt && echo "requirements found" || echo "requirements NOT found"

# 3) show python interpreter đang dùng
import sys
print("Python executable:", sys.executable)

# 4) dùng chính interpreter đó để cài (an toàn hơn dùng !pip trực tiếp)
!{sys.executable} -m pip install -r vlsp-mt/requirements.txt

/content/BTL-NLP-2526I_INT3406_3
/content/BTL-NLP-2526I_INT3406_3
total 24
drwxr-xr-x 5 root root 4096 Dec 15 08:38 .
drwxr-xr-x 1 root root 4096 Dec 15 08:38 ..
drwxr-xr-x 8 root root 4096 Dec 15 08:38 .git
-rw-r--r-- 1 root root   25 Dec 15 08:38 README.md
drwxr-xr-x 2 root root 4096 Dec 15 08:38 transformer_base
drwxr-xr-x 4 root root 4096 Dec 15 08:38 vlsp-mt
total 36
drwxr-xr-x 4 root root  4096 Dec 15 08:38 .
drwxr-xr-x 5 root root  4096 Dec 15 08:38 ..
drwxr-xr-x 3 root root  4096 Dec 15 08:38 data
-rw-r--r-- 1 root root     0 Dec 15 08:38 experiment_log.md
-rw-r--r-- 1 root root   485 Dec 15 08:38 requirements.txt
-rw-r--r-- 1 root root 13116 Dec 15 08:38 run.md
drwxr-xr-x 2 root root  4096 Dec 15 08:38 scripts
requirements found
Python executable: /usr/bin/python3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 k

# Run script

In [ ]:
! ls -a

.  ..  .git  README.md	transformer_base  vlsp-mt


In [ ]:
%cd vlsp-mt

/content/BTL-NLP-2526I_INT3406_3/vlsp-mt


In [ ]:
!python scripts/dedup_minhash.py \
    --src data/raw/train.en.txt \
    --tgt data/raw/train.vi.txt \
    --out_dir data/dedup \
    --threshold 0.8 \
    --dedup_by both \
    --rep_strategy longest

MinHash LSH Deduplication

Loading data...
Loaded 500,000 pairs
Preparing texts for hashing...
Building MinHash signatures (threshold=0.8, k=5)...
Using 12 workers for parallel processing...
Computing MinHash: 100% 500000/500000 [05:30<00:00, 1510.65it/s]
Building LSH index...
Indexing: 100% 500000/500000 [00:16<00:00, 29867.11it/s]
Finding duplicate clusters...
Clustering: 100% 500000/500000 [00:10<00:00, 49161.03it/s]

DEDUPLICATION RESULTS
  Original pairs:     500,000
  Kept pairs:         342,789
  Removed pairs:      157,211
  Keep ratio:         68.56%
  Number of clusters: 342,789
  Avg cluster size:   1.46
  Max cluster size:   187
  Singleton clusters: 190,929

Saving deduplicated data...
  Source: data/dedup/train.en
  Target: data/dedup/train.vi
  Metadata: data/dedup/dedup_meta.json


In [ ]:
!python scripts/preprocess_vlsp.py \
    --src_in data/dedup/train.en \
    --tgt_in data/dedup/train.vi \
    --out_dir data/clean \
    --min_len 3 \
    --max_len 256 \
    --max_ratio 3.0 \
    --dev_size 1000 \
    --test_size 1000

VLSP Medical Translation Preprocessing

Loading data from:
  Source: data/dedup/train.en
  Target: data/dedup/train.vi
Loaded 342,789 pairs

Cleaning...
After cleaning: 342,789 pairs

Filtering...
After filtering: 334,510 pairs

Deduplicating by 'both'...
After dedup: 334,510 pairs

Splitting (dev=1000, test=1000)...

Train:
  Pairs: 332,510
  Src length: min=3, max=229, avg=21.3
  Tgt length: min=3, max=256, avg=30.2

Dev:
  Pairs: 1,000
  Src length: min=3, max=125, avg=21.8
  Tgt length: min=3, max=210, avg=30.9

Test:
  Pairs: 1,000
  Src length: min=3, max=146, avg=20.7
  Tgt length: min=5, max=243, avg=29.3

Preprocessing complete!
Output saved to: data/clean
Config saved to: data/clean/preprocess_config.json


In [ ]:
%cd ../

/content/BTL-NLP-2526I_INT3406_3


In [ ]:
!ls -a

.  ..  .git  README.md	transformer_base  vlsp-mt


In [ ]:
# 1. Nén thư mục 'vlsp-mt/data' thành file 'data_backup.zip'

!zip -r data_backup.zip vlsp-mt

  adding: vlsp-mt/ (stored 0%)
  adding: vlsp-mt/scripts/ (stored 0%)
  adding: vlsp-mt/scripts/preprocess_vlsp.py (deflated 69%)
  adding: vlsp-mt/scripts/dedup_minhash.py (deflated 68%)
  adding: vlsp-mt/scripts/rl_train_grpo.py (deflated 70%)
  adding: vlsp-mt/scripts/train_qwen_lora.py (deflated 72%)
  adding: vlsp-mt/scripts/generate.py (deflated 70%)
  adding: vlsp-mt/scripts/run_eval_all.py (deflated 67%)
  adding: vlsp-mt/scripts/eval_bleu.py (deflated 68%)
  adding: vlsp-mt/run.md (deflated 72%)
  adding: vlsp-mt/outputs/ (stored 0%)
  adding: vlsp-mt/outputs/dev.hyp.sft.vi (deflated 69%)
  adding: vlsp-mt/outputs/dev.hyp.rl.vi (deflated 68%)
  adding: vlsp-mt/runs/ (stored 0%)
  adding: vlsp-mt/runs/qwen_fast/ (stored 0%)
  adding: vlsp-mt/runs/qwen_fast/training_history.json (deflated 67%)
  adding: vlsp-mt/runs/qwen_fast/checkpoint-5195/ (stored 0%)
  adding: vlsp-mt/runs/qwen_fast/checkpoint-5195/training_args.bin (deflated 53%)
  adding: vlsp-mt/runs/qwen_fast/checkpoint-

In [ ]:
# 2. Tải file zip về máy
from google.colab import files
files.download('data_backup.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# 1. Tạo thư mục đích trước
!mkdir -p data/rl_subset

# 2. Chạy lệnh cắt file
!head -n 50000 data/clean/train.en > data/rl_subset/en.txt
!head -n 50000 data/clean/train.vi > data/rl_subset/vi.txt

!ls -lh data/rl_subset/

total 16M
-rw-r--r-- 1 root root 6.9M Dec 15 08:45 en.txt
-rw-r--r-- 1 root root 8.7M Dec 15 08:45 vi.txt


In [ ]:
!python scripts/train_qwen_lora.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --direction en2vi \
    --src data/clean/train.en \
    --tgt data/clean/train.vi \
    --run_id qwen_fast \
    --batch_size 32 \
    --grad_accum 1 \
    --epochs 1 \
    --neftune_alpha 0 \
    --no_grad_checkpoint \
    --label_smoothing 0


2025-12-15 08:45:46.803670: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 08:45:46.821753: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765788346.843772    3066 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765788346.850277    3066 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765788346.866815    3066 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!head -n 10000 data/clean/train.en > data/rl_subset/en.txt
!head -n 10000 data/clean/train.vi > data/rl_subset/vi.txt

In [ ]:
!python scripts/rl_train_grpo.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --sft_adapter runs/qwen_fast/lora_en2vi_sft \
    --init_adapter runs/qwen_fast/lora_en2vi_sft \
    --rl_src data/rl_subset/en.txt \
    --rl_tgt data/rl_subset/vi.txt \
    --run_id qwen_rl \
    --direction en2vi \
    --batch_size 16 \
    --max_new_tokens 48

2025-12-15 12:15:22.432700: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 12:15:22.452392: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765800922.475129   54614 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765800922.481866   54614 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765800922.499311   54614 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/qwen_fast/lora_en2vi_sft \
    --direction en2vi \
    --input data/clean/dev.en \
    --output outputs/dev.hyp.sft.vi \
    --batch_size 8

2025-12-15 15:03:02.885671: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 15:03:02.904220: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765810982.926718   96287 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765810982.933361   96287 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765810982.950247   96287 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/generate.py \
    --model_name Qwen/Qwen2.5-3B-Instruct \
    --adapter_path runs/qwen_rl/final_model \
    --direction en2vi \
    --input data/clean/dev.en \
    --output outputs/dev.hyp.rl.vi \
    --batch_size 8

2025-12-15 15:27:00.768243: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-15 15:27:00.786652: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765812420.808712  102245 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765812420.815192  102245 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1765812420.831986  102245 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [ ]:
!python scripts/eval_bleu.py \
    --hyp outputs/dev.hyp.sft.vi \
    --ref data/clean/dev.vi

Loading hypothesis: outputs/dev.hyp.sft.vi
Loading reference: data/clean/dev.vi

Evaluating 1000 sentence pairs...

EVALUATION RESULTS
  BLEU:    49.84
  chrF++:  64.40
  chrF:    64.52
  TER:     42.31 (lower is better)


In [ ]:
!python scripts/eval_bleu.py --hyp outputs/dev.hyp.rl.vi --ref data/clean/dev.vi

Loading hypothesis: outputs/dev.hyp.rl.vi
Loading reference: data/clean/dev.vi

Evaluating 1000 sentence pairs...

EVALUATION RESULTS
  BLEU:    50.09
  chrF++:  64.65
  chrF:    64.78
  TER:     42.28 (lower is better)
